In [1]:
import pandas as pd
import numpy as np
import requests
import pendulum
from datetime import tzinfo, timedelta, datetime, timezone, time

In [2]:
def download_image_url(date, day_night_ind, bbox, latitude, longitude):
    """
    example arguments:
     time: datetime(2020-01-29T12:10:00Z)
     bbox: [10.177734375, -60.1875, 16.41796875, -53.5166015625]
     layers: ["GOES-East_ABI_Band2_Red_Visible_1km", "Reference_Labels"]
     image_format: "tiff"
     resolution: 0.01 (deg/pixel)

     `bbox` should be a list of [lat_S, lon_W, lat_N, lon_E]
     
    """
    BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME={datetimestamp}&WRAP=DAY,DAY&BBOX={bbox}&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts={timestamp}"

    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]

    hour = 12
    if day_night_ind == "N":
        hour = 22
    
    dt = datetime(int(year), int(month), int(day), hour, 0, 0, 0, tzinfo=timezone.utc)
    #dt = pendulum.instance(dt).to_iso8601_string()
    print(dt)
    

    url = BASE_URL.format(
        datetimestamp=date,
        bbox=",".join([str(v) for v in bbox]),
        timestamp = datetime.timestamp(dt)
    )

    print("url : "+url)
    return url
    

def download_img(date, day_night_ind, bbox, latitude, longitude):
    url = download_image_url(date, day_night_ind, bbox, latitude, longitude)
    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]
    file_name = "./satellite_image/"+year+'/'+str(longitude)+'_'+str(latitude)+'-'+date+'-'+day_night_ind+'.png'
    #print(url)
    r = requests.get(url)
    if r.status_code == 200:
        if 'xml' in r.text[:40]:
            print(url)
            raise Exception(r.content)
        else:
            with open(file_name, 'wb') as fh:
                fh.write(r.content)
    else:
        raise Exception(r.status)



In [3]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    one_downloaded = 0
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+5
            south_coord = latitude-5
            west_coord = longitude-5
            east_coord = longitude+5
            area = [south_coord, west_coord, north_coord, east_coord]
            #area = [44.1105,-80.8925,63.4595,-55.9775]
            download_img(date_value, day_night_ind, area, latitude, longitude )


In [4]:
#group_fire_data_xarray("canada_2023")

#group_fire_data_xarray("canada_2021")

2023-01-01 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-01&WRAP=DAY,DAY&BBOX=48.086800000000004,-122.31796666666666,58.086800000000004,-112.31796666666666&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1672574400.0
2023-01-01 22:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-01&WRAP=DAY,DAY&BBOX=49.41256666666666,-122.544,59.41256666666666,-112.544&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1672610400.0
2023-01-02 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-02&WRAP=DAY,DAY&BBOX=48.4962,-122.966975,58.4962,-112.966975

In [5]:
group_fire_data_xarray("canada_2022")

2022-01-11 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2022-01-11&WRAP=DAY,DAY&BBOX=50.339499999999994,-125.7944,60.339499999999994,-115.7944&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1641902400.0
2022-01-12 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2022-01-12&WRAP=DAY,DAY&BBOX=49.6095,-120.60915,59.6095,-110.60915&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1641988800.0
2022-01-13 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2022-01-13&WRAP=DAY,DAY&BBOX=49.606750000000005,-123.8242,59.606750000000005,-113.8242&FORMAT=image/png&

Exception: b'<?xml version=\'1.0\' encoding="UTF-8" standalone="no" ?>\n<ServiceExceptionReport version="1.3.0" xmlns="http://www.opengis.net/ogc" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/ogc http://schemas.opengis.net/wms/1.3.0/exceptions_1_3_0.xsd">\n<ServiceException>\nmsDrawMap(): WMS connection error. Failed to draw WMS layer named &#39;MODIS_Aqua_CorrectedReflectance_TrueColor&#39;. This most likely happened because the remote WMS server returned an invalid image, and XML exception or another unexpected result in response to the GetMap request. Also check and make sure that the layer&#39;s connection URL is valid.\ndrawGDAL(): Unable to access file. GDALDatasetRasterIO() failed: Error while reading row 878: libpng: Read Error\n</ServiceException>\n</ServiceExceptionReport>\n'